In [25]:
import itertools
import qutip
import numpy as np
import theano
import theano.tensor as T
import theano.tensor.slinalg
import theano.tensor.nlinalg

In [2]:
def complexrandn(dim1, dim2):
    big_matrix = np.random.randn(dim1, dim2, 2)
    return big_matrix[:, :, 0] + 1.j * big_matrix[:, :, 1]

def complex2bigreal(matrix):
    row1 = np.concatenate((np.real(matrix), -np.imag(matrix)), axis=1)
    row2 = np.concatenate((np.imag(matrix), np.real(matrix)), axis=1)
    return np.concatenate((row1, row2), axis=0)

In [3]:
H = complexrandn(2, 2)
# print(H)
Hp1 = np.concatenate((-np.imag(H), -np.real(H)), axis=1)
Hp2 = np.concatenate((np.real(H), -np.imag(H)), axis=1)
Hp = np.concatenate((Hp1, Hp2), axis=0)
x = T.dscalar('x')
expH = T.slinalg.expm(x * Hp)
# theano.pp(expH)
expH_flat = T.flatten(expH)
def fn(i, M, x):
    return T.grad(M[i], x)
J, updates = theano.scan(fn=fn,
                         sequences=[T.arange(expH_flat.shape[0])],
                         non_sequences=[expH_flat, x])
gexpH = J.reshape(expH.shape)
f = theano.function([x], gexpH)
f(2.)

array([[ -11.98245409,   46.98510923,   15.2539712 ,   38.72437155],
       [  44.44146309, -126.62033605,  -40.91030224, -145.83578874],
       [ -15.2539712 ,  -38.72437155,  -11.98245409,   46.98510923],
       [  40.91030224,  145.83578874,   44.44146309, -126.62033605]])

In [6]:
# get_sigmas_index gets a tuple as input and gives back
# a length-16 array of zeros with only one element equal to 1
def get_sigmas_index(indices):
    all_zeros = np.zeros(4 * 4)
    all_zeros[indices[0] * 4 + indices[1]] = 1.
    return all_zeros

# generate all tensor products of sigmas
sigmas = [qutip.qeye(2), qutip.sigmax(), qutip.sigmay(), qutip.sigmaz()]
sigma_pairs = []
for idx1 in range(4):
    for idx2 in range(4):
        sigma_pairs.append(
            complex2bigreal(
                1j * qutip.tensor(sigmas[idx1], sigmas[idx2]).data.toarray()))
sigma_pairs = np.asarray(sigma_pairs)

print(sigma_pairs.shape)

# J is the theano vector containing all the interactions strengths
J = T.dvector('J')
H = T.tensordot(J, sigma_pairs, axes=1)

expH = T.slinalg.expm(H)

f = theano.function([J], H)
f(get_sigmas_index((0, 1)))
# theano.printing.pydotprint(H, 'testPNG.png')

(16, 8, 8)


array([[ 0.,  0.,  0.,  0.,  0., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]])

In [7]:
import qutip
import itertools
from collections import OrderedDict

def chars2pair(chars):
    out_pair = []
    for idx in range(len(chars)):
        if chars[idx] == 'x':
            out_pair.append(1)
        elif chars[idx] == 'y':
            out_pair.append(2)
        elif chars[idx] == 'z':
            out_pair.append(3)
        else:
            raise ValueError('chars must contain 2 characters, each of'
                             'which equal to either x, y, or z')
    return tuple(out_pair)

class QubitNetwork:
    def __init__(self, num_qubits,
                 interactions='all', self_interactions='all',
                 system_qubits=None):
        # *self.num_qubits* is the TOTAL number of qubits in the networl,
        # regardless of them being system or ancilla qubits
        self.num_qubits = num_qubits
        # we store all the possible pairs for convenience
        self.pairs = list(itertools.combinations(range(self.num_qubits), 2))
        # decode_interactions_dict fills the self.active_Js variable
        self.active_Js = self.decode_interactions(interactions)
        self.active_hs = self.decode_self_interactions(self_interactions)
        
        self.num_interactions = self.count_interactions()
        self.num_self_interactions = self.count_self_interactions()
        
        # Js_factors and hs_factors store the matrices, in big real form,
        # that will have to multiplied by the *Js* and *hs* factors
        self.Js_factors, self.hs_factors = self.build_H_components()
        
        # the initial state is here build in big real form, with all
        # the qubits initialized in the up position
        self.initial_state = self.build_initial_state_vector()
        
        # Define which qubits belong to the system. The others are all
        # assumed to be ancilla qubits. If *system_qubits* was not explicitly
        # given it is assumed that half of the qubits are the system and half
        # are ancillas
        if system_qubits is None:
            self.system_qubits = tuple(range(num_qubits // 2))
        elif np.all(np.asarray(system_qubits) < num_qubits):
            self.system_qubits = tuple(system_qubits)
        else:
            raise ValueError('Invalid value for system_qubits.')

    def decode_interactions(self, interactions):
        if interactions == 'all':
            allsigmas = [item[0] + item[1]
                         for item in itertools.product(['x', 'y', 'z'], repeat=2)]
            return OrderedDict([(pair, allsigmas) for pair in self.pairs])
        elif isinstance(interactions, tuple):
            if interactions[0] == 'all':
                d = {pair: interactions[1] for pair in self.pairs}
                return OrderedDict(d)
        elif (isinstance(interactions, dict) and
              all(isinstance(k, tuple) for k in interactions.keys())):
            return OrderedDict(interactions)
        else:
            raise ValueError('Invalid value given for interactions.')
    
    def decode_self_interactions(self, self_interactions):
        if self_interactions == 'all':
            return OrderedDict(
                {idx: ['x', 'y', 'z'] for idx in range(self.num_qubits)})
        elif isinstance(self_interactions, tuple):
            if self_interactions[0] == 'all':
                d = {idx: self_interactions[1] for idx in range(self.num_qubits)}
                return OrderedDict(d)
            else:
                raise ValueError('Invalid value for self_interactions.')
        else:
            raise ValueError('Invalid value of self_interactions.')
            
    
    def count_interactions(self):
        count = 0
        for k, v in self.active_Js.items():
            count += len(v)
        return count

    def count_self_interactions(self):
        count = 0
        for k, v in self.active_hs.items():
            count += len(v)
        return count
    
    def build_H_components(self):
        terms_template = [qutip.qeye(2) for _ in range(self.num_qubits)]
        Js_factors = []
        hs_factors = []
        for pair, directions in self.active_Js.items():
            # - directions is a list of elements like ss below
            # - ss is a two-character string specifying an interaction
            # direction, e.g. 'xx' or 'xy' or 'zy'
            for ss in directions:
                term = terms_template
                term[pair[0]] = sigmas[chars2pair(ss)[0]]
                term[pair[1]] = sigmas[chars2pair(ss)[1]]
                term = complex2bigreal(1j * qutip.tensor(term).data.toarray())
                Js_factors.append(term)

        for qubit_idx, direction in self.active_hs.items():
            # - now direction is a list of characters among 'x', 'y' and 'z',
            # - s is either 'x', 'y', or 'z'
            for s in direction:
                term = terms_template
                term[qubit_idx] = sigmas[chars2pair(s)[0]]
                term = complex2bigreal(1j * qutip.tensor(term).data.toarray())
                hs_factors.append(term)

        return np.asarray(Js_factors), np.asarray(hs_factors)
    
    def build_initial_state_vector(self):
        state = qutip.tensor([qutip.basis(2, 0) for _ in range(self.num_qubits)])
        state = state.data.toarray()
        state = np.concatenate((np.real(state), np.imag(state)), axis=0)
        return state

In [21]:
net = QubitNetwork(4, interactions=('all', ['zz']),
                   self_interactions=('all', ['x', 'y']),
                   system_qubits=[0, 1, 2])
net.num_qubits

4

In [36]:
# net.hs_factors and net.Js_factors are already in big real matrix form,
# and already multiplied by 1j
H_factors = np.concatenate((net.hs_factors, net.Js_factors), axis=0)
# J is the theano vector containing all the interactions strengths and self energies
J = T.dvector('J')
# this builds the Hamiltonian of the system (in big real matrix form),
# already multiplied with the 1j factor and ready for exponentiation
H = T.tensordot(J, H_factors, axes=1)
# expH is the unitary evolution of the system
expH = T.slinalg.expm(H)

# net.initial_state is the initial state stored as a vector (ket),
# multiplying it on the left by expH amounts to evolve it
expH_times_state = T.dot(expH, net.initial_state)
# build the density matrix out of the evolved state
dm = expH_times_state * expH_times_state.T
dm_real = dm[0:dm.shape[0] // 2, 0:dm.shape[1] // 2]
dm_imag = dm[0:dm.shape[0] // 2, dm.shape[1] // 2:]
# partial trace
def row_fn(row, matrix):
    res, updates = theano.scan(fn=col_fn,
                               sequences=T.arange(matrix.shape[1]))
    return res

# dm2 = T.nlinalg.trace(dm[1:9, 1:9])

# expH_flat = T.flatten(expH)
# def fn(i, matrix, x):
#     return T.grad(matrix[i], x)
# expH_flat_grad, updates = theano.scan(fn=fn,
#                                       sequences=T.arange(expH_flat.shape[0]),
#                                       non_sequences=[expH_flat, J])

# grads = theano.function([J], expH_flat_grad.reshape(expH.shape))

# generate random initial parameters
initial_parameters = np.random.randn(net.num_interactions + net.num_self_interactions)
f = theano.function([J], dm_real)
f(initial_parameters)

array([[  3.03556143e-03,   1.66358705e-03,   4.07523580e-03,
          6.19026651e-04,  -3.01127481e-04,  -8.74648854e-04,
         -7.70966682e-03,  -2.31993998e-03,  -5.22443454e-04,
         -5.65659741e-03,   2.55806487e-03,   5.50443239e-04,
          1.22820288e-03,   1.29057079e-02,  -3.25598837e-03,
          3.18211909e-02],
       [  1.66358705e-03,   9.11700173e-04,   2.23336264e-03,
          3.39246873e-04,  -1.65027719e-04,  -4.79336209e-04,
         -4.22514984e-03,  -1.27140307e-03,  -2.86316118e-04,
         -3.10000059e-03,   1.40190330e-03,   3.01660917e-04,
          6.73095393e-04,   7.07275048e-03,  -1.78438823e-03,
          1.74390545e-02],
       [  4.07523580e-03,   2.23336264e-03,   5.47099677e-03,
          8.31042172e-04,  -4.04263104e-04,  -1.17421452e-03,
         -1.03502139e-02,  -3.11451528e-03,  -7.01379403e-04,
         -7.59397192e-03,   3.43419752e-03,   7.38969064e-04,
          1.64886017e-03,   1.73258898e-02,  -4.37115856e-03,
          4.2719

In [32]:
H = T.dmatrix('H')
def fn2(col, row, matrix):
    return matrix[row, col] + row * col

def fn(row, matrix):
    res, updates = theano.scan(fn=fn2,
                               sequences=T.arange(matrix.shape[1]),
                               non_sequences=[row, matrix])
    return res

results, updates = theano.scan(fn=fn,
                               sequences=T.arange(H.shape[0]),
                               non_sequences=[H])
f = theano.function([H], results)
f(np.arange(9).reshape(3, 3))

array([[  0.,   1.,   2.],
       [  3.,   5.,   7.],
       [  6.,   9.,  12.]])